In [1]:
%load_ext autoreload
%autoreload 2
%run ~/FKMC/notebooks/notebook_preamble.py
%matplotlib inline
plt.style.use('../custom.mplstyle')

import multiprocessing as mp

np.seterr(under = 'ignore')
data_location = Path('~/HPC_data/pickled_data').expanduser()

Imported pathlib::Path, os, munch::Munch, pickle, logging, itertools::count, matplotlib::colors
Names are pyplot = plt, matplotlib = mpl, numpy = np


In [4]:
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
from itertools import product, islice
from FKMC.montecarlo import *

## overall input parameters
T = 2.5

Ns = np.logspace(np.log10(50), np.log10(270), 10, dtype = np.int) // 10 * 10
Ns = np.array([10,20,30,40,50])

make_initial_state = lambda N: np.arange(N, dtype = np.float64) % 2 #a CDW initial state
J = 5
U = 5
alpha = 1.25
jobdir = Path('~/HPC_data/local_run_data/fermion_order_parameter').expanduser()
jobdir.mkdir(parents=False, exist_ok=True)

logs = np.empty(shape = len(Ns), dtype = object)
previous_states = [make_initial_state(N) for N in Ns]
t1 = time()

for j in range(1):
    print(f"On round j = {j}")
    for i, N in enumerate(Ns):
        print(f'Starting N = {N}')
        parameters = dict(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
        MCMC_params = dict(
                state = previous_states[i],
                N_steps = int(10000),
                N_burn_in = int(0),
                thin = 10,
                logger = all_and_eigenvects(bins = 10000, limit = 20),
                proposal = p_one_or_two,
                accept_function = perturbation_accept,
                warnings = True,
            )

        t0 = time()
        logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
        logs[i].time = time() - t0
        
        previous_states[i] = logs[i].last_state

        print(f'This N = {N} j ={i} took {time() - t0:.0f} seconds.')

        filepath = jobdir/f'{j}.npz'
        print(f'Saving in {filepath}')
        np.savez_compressed(filepath, 
        parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True, Ns = Ns,
        )

    print(f'Overall: {time() - t1:.0f} seconds.\n\n')


On round j = 0
Starting N = 10
N = 10: 0% through after 0.00m             acceptance rates: classical = 0% quantum = 0% overall = 0%
N = 10: 10% through after 0.00m             acceptance rates: classical = 36% quantum = 55% overall = 20%
N = 10: 20% through after 0.01m             acceptance rates: classical = 42% quantum = 57% overall = 24%
N = 10: 30% through after 0.01m             acceptance rates: classical = 44% quantum = 57% overall = 25%
N = 10: 40% through after 0.02m             acceptance rates: classical = 49% quantum = 57% overall = 28%
N = 10: 50% through after 0.02m             acceptance rates: classical = 48% quantum = 57% overall = 27%
N = 10: 60% through after 0.02m             acceptance rates: classical = 48% quantum = 58% overall = 28%
N = 10: 70% through after 0.03m             acceptance rates: classical = 50% quantum = 57% overall = 28%
N = 10: 80% through after 0.03m             acceptance rates: classical = 49% quantum = 57% overall = 27%
N = 10: 90% through

In [ ]:
d = np.load(filepath)